In [15]:
import boule as bl
import harmonica as hm
import verde as vd
import numpy as np

In [65]:
topography = hm.datasets.fetch_topography_earth().loc[dict(longitude=slice(-180, 180, 10), latitude=slice(-90, 90, 10))]
topography

<xarray.Dataset>
Dimensions:     (latitude: 37, longitude: 73)
Coordinates:
  * longitude   (longitude) float64 -180.0 -175.0 -170.0 ... 170.0 175.0 180.0
  * latitude    (latitude) float64 -90.0 -85.0 -80.0 -75.0 ... 80.0 85.0 90.0
Data variables:
    topography  (latitude, longitude) float64 2.762e+03 2.762e+03 ... -4.179e+03

In [66]:
ellipsoid = bl.WGS84

In [67]:
topo = vd.grid_to_table(topography)

In [68]:
dlats = topography.latitude.values[1:] - topography.latitude.values[:-1]
dlons = topography.longitude.values[1:] - topography.longitude.values[:-1]
dlon = dlons[0]
dlat = dlats[0]
assert np.allclose(dlons, dlon)
assert np.allclose(dlats, dlat)

In [83]:
west = topography.longitude.values - dlon/2
east = topography.longitude.values + dlon/2
west[west > 360] = 360
west[west < -180] = -180
east[east > 360] = 360
east[east < -180] = -180
south = topography.latitude.values - dlat/2
south[south > 90] = 90
south[south < -90] = -90
north = topography.latitude.values + dlat/2
north[north > 90] = 90
north[north < -90] = -90
reference = ellipsoid.geodetic_to_spherical(None, topo.latitude, np.zeros_like(topo.latitude))[2]
radius = ellipsoid.geodetic_to_spherical(None, topo.latitude, topo.topography)[2]
index = np.arange(radius.size)

In [84]:
coordinates = ellipsoid.geodetic_to_spherical(topo.longitude, topo.latitude, np.full_like(topo.latitude, 100e3))

In [85]:
zero_height = np.zeros_like(north)
north_sph = ellipsoid.geodetic_to_spherical(None, north, zero_height)[1]
south_sph = ellipsoid.geodetic_to_spherical(None, south, zero_height)[1]

In [86]:
shape = topography.topography.shape

In [87]:
batch_size = 10
batches = [index[i:i + batch_size] for i in range(0, index.size, batch_size)]
field = np.zeros_like(topo.topography)
for batch in batches:
    tesseroids = []
    for t in batch:
        i, j = np.unravel_index(t, shape)
        tesseroids.append([west[j], east[j], south[i], north[i], reference[t], radius[t]])
    density = np.full_like(batch, 2670)
    field += hm.tesseroid_gravity(coordinates, tesseroids, field="g_z", density=density)
    

ValueError: Invalid tesseroid or tesseroids. The bottom radius boundary can't be greater than the top one.
	Invalid tesseroid: [-1.80000000e+02 -1.77500000e+02 -8.25000000e+01 -7.75000000e+01
  6.35691611e+06  6.35682711e+06]
	Invalid tesseroid: [-1.77500000e+02 -1.72500000e+02 -8.25000000e+01 -7.75000000e+01
  6.35675231e+06  6.35673831e+06]


In [79]:
np.unravel_index(batches[0], shape)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))